In [1]:
import warnings

import pandas as pd
import torch
from datasets import Dataset, load_dataset
from tqdm import tqdm, tqdm_notebook
from transformers import AutoModelForSequenceClassification, AutoTokenizer, HfArgumentParser
from trl import ModelConfig, RewardConfig, RewardTrainer
from itertools import product 

c:\ml_proj\tink_stag\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
#Формирование датасета для обучения модели вознаграждения

raw_datasets = load_dataset("imdb", split="train")

df = pd.DataFrame(data=raw_datasets)
positive = df.text[df['label'] == 0]
negative = df.text[df['label'] == 1]
new_df = pd.DataFrame({"positive": positive.to_list(), "negative": negative.to_list()})
dataset = Dataset.from_pandas(new_df)


In [3]:
dataset

Dataset({
    features: ['positive', 'negative'],
    num_rows: 12500
})

In [4]:
tokenizer = AutoTokenizer.from_pretrained("distilbert-base-cased")

In [5]:
#Преобразования для вида, необходимого RewardTrainer

def preprocess_function(examples):
        new_examples = {
            "input_ids_chosen": [],
            "attention_mask_chosen": [],
            "input_ids_rejected": [],
            "attention_mask_rejected": [],
        }

       
        for pos, neg in zip(examples["positive"], examples["negative"]):
            tokenized_pos = tokenizer(pos)
            tokenized_neg = tokenizer(neg)

            new_examples["input_ids_chosen"].append( tokenized_pos["input_ids"])
            new_examples["attention_mask_chosen"].append( tokenized_pos["attention_mask"])
            new_examples["input_ids_rejected"].append(tokenized_neg["input_ids"])
            new_examples["attention_mask_rejected"].append(tokenized_neg["attention_mask"])

                
        return new_examples

In [6]:
dataset = dataset.map(preprocess_function,
        batched=True,
    )

Map: 100%|██████████| 12500/12500 [00:09<00:00, 1354.22 examples/s]


In [7]:
# Preprocess the dataset and filter out examples that are longer than args.max_length

dataset = dataset.filter(
    lambda x: len(x["input_ids_chosen"]) <= 512 and len(x["input_ids_rejected"]) <= 512
    )


Filter: 100%|██████████| 12500/12500 [00:03<00:00, 3810.98 examples/s]


In [8]:
dataset

Dataset({
    features: ['positive', 'negative', 'input_ids_chosen', 'attention_mask_chosen', 'input_ids_rejected', 'attention_mask_rejected'],
    num_rows: 8891
})

In [9]:
"""Все возможные пары

def pairs(dataset):
    
    positive = dataset["input_ids_chosen"]
    negative = dataset["input_ids_rejected"]
    positive_mask = dataset["attention_mask_chosen"]
    negative_mask = dataset["attention_mask_rejected"]
    pairs_ids = (list(product(positive, negative)))
    pairs_mask = (list(product(positive_mask, negative_mask)))
    input_ids_chosen, input_ids_rejected = map(list, zip(*pairs_ids))
    attention_mask_chosen, attention_mask_rejected = map(list, zip(*pairs_mask))
    df_com = pd.DataFrame({"input_ids_chosen": input_ids_chosen, "input_ids_rejected": input_ids_rejected,
                       "attention_mask_chosen": attention_mask_chosen, "attention_mask_rejected": attention_mask_rejected})

"""

'Все возможные пары\n\ndef pairs(dataset):\n    \n    positive = dataset["input_ids_chosen"]\n    negative = dataset["input_ids_rejected"]\n    positive_mask = dataset["attention_mask_chosen"]\n    negative_mask = dataset["attention_mask_rejected"]\n    pairs_ids = (list(product(positive, negative)))\n    pairs_mask = (list(product(positive_mask, negative_mask)))\n    input_ids_chosen, input_ids_rejected = map(list, zip(*pairs_ids))\n    attention_mask_chosen, attention_mask_rejected = map(list, zip(*pairs_mask))\n    df_com = pd.DataFrame({"input_ids_chosen": input_ids_chosen, "input_ids_rejected": input_ids_rejected,\n                       "attention_mask_chosen": attention_mask_chosen, "attention_mask_rejected": attention_mask_rejected})\n\n'

In [10]:
train_on_gpu = torch.cuda.is_available()

if not train_on_gpu:
    print('CUDA is not available.  Training on CPU ...')
else:
    print('CUDA is available!  Training on GPU ...')

CUDA is available!  Training on GPU ...


In [22]:
dataset = dataset.train_test_split(test_size=0.1)

In [14]:
model = AutoModelForSequenceClassification.from_pretrained("distilbert-base-cased", num_labels=1)

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-cased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [15]:
model = model.to("cuda")

In [16]:
config = {"max_length": None,
          "output_dir": "reward_modeling_anthropic_hh",
          "per_device_train_batch_size": 16,
          "num_train_epochs": 3,
          "gradient_accumulation_steps": 2,
          "gradient_checkpointing": True,
          "learning_rate": 1.41e-5,
          "remove_unused_columns": False,
          "optim": "adamw_torch",
          "logging_steps": 10}

In [17]:
import warnings
warnings.filterwarnings(action='ignore', category=DeprecationWarning)


In [18]:
parser = HfArgumentParser((RewardConfig, ModelConfig))
config, model_config = parser.parse_dict(config)
device = "cuda"


In [23]:
trainer = RewardTrainer(
    model=model,
    args = config,
    tokenizer=tokenizer,
    train_dataset=dataset["train"],
    eval_dataset=dataset["test"]
)

# train
trainer.train()

c:\ml_proj\tink_stag\.venv\Lib\site-packages\trl\trainer\reward_trainer.py:175: UserWarning: When using RewardDataCollatorWithPadding, you should set `max_length` in RewardConfig. It will be set to `512` by default, but you should do it yourself in the future.
  warnings.warn(
  0%|          | 0/750 [00:00<?, ?it/s]c:\ml_proj\tink_stag\.venv\Lib\site-packages\transformers\tokenization_utils_base.py:2906: UserWarning: `max_length` is ignored when `padding`=`True` and there is no truncation strategy. To pad to max length, use `padding='max_length'`.
  warnings.warn(
c:\ml_proj\tink_stag\.venv\Lib\site-packages\torch\utils\checkpoint.py:464: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.4 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between 

{'loss': 0.0437, 'grad_norm': 1.0831670761108398, 'learning_rate': 1.3912000000000002e-05, 'epoch': 0.04}


  3%|▎         | 20/750 [00:34<19:52,  1.63s/it]

{'loss': 0.0801, 'grad_norm': 3.3010365962982178, 'learning_rate': 1.3724000000000001e-05, 'epoch': 0.08}


  4%|▍         | 30/750 [00:51<20:51,  1.74s/it]

...

100%|██████████| 750/750 [21:28<00:00,  1.69s/it]

{'loss': 0.0041, 'grad_norm': 0.7030878663063049, 'learning_rate': 0.0, 'epoch': 2.99}


100%|██████████| 750/750 [21:32<00:00,  1.72s/it]

{'train_runtime': 1292.1564, 'train_samples_per_second': 18.576, 'train_steps_per_second': 0.58, 'train_loss': 0.027654054552316665, 'epoch': 2.99}


TrainOutput(global_step=750, training_loss=0.027654054552316665, metrics={'train_runtime': 1292.1564, 'train_samples_per_second': 18.576, 'train_steps_per_second': 0.58, 'total_flos': 0.0, 'train_loss': 0.027654054552316665, 'epoch': 2.9940119760479043})

In [49]:
model.save_pretrained('reward_model')